# EM update for prior variance when prior cannot be directly inverted

The [EM approach previously shown](elbo_prior_est.html) for estimating prior variance scalar has a problem: its current form does not work when input prior matrices has non-invertable component. I used generalized inverse in the code but it does not work.

## Illustration

In [1]:
devtools::load_all('~/GIT/software/mmbr')
set.seed(1)
dat = mmbr_sim1(n=50,p=50,r=2,s=2)
L = 10

Loading mmbr

Loading required package: mashr

Loading required package: ashr



In [2]:
V = matrix(1,2,2)

In [31]:
fit = msusie(dat$X,dat$y,L=L,prior_variance=V, 
              compute_objective=T, estimate_residual_variance=F, 
              estimate_prior_variance=T, estimate_prior_method = 'EM',
              track_fit=TRUE)

Loading mmbr

Warning message in m$get_objective(dump = TRUE):
“Objective is not non-decreasing”


In [32]:
fit$elbo

[1] -198.1840 -185.4796 -184.7428 -184.7332 -184.8186

In [33]:
fit$prior_history

[[1]]
 [1] 0.14162026 0.03844848 0.02970158 0.02592655 0.02361152 0.02201506
 [7] 0.02083379 0.01991772 0.01918332 0.01857995

[[2]]
 [1] 0.077112125 0.010839044 0.009555124 0.008928328 0.008507832 0.008196177
 [7] 0.007950441 0.007748364 0.007577150 0.007428863

[[3]]
 [1] 0.044767523 0.004897835 0.004378688 0.004126129 0.003957653 0.003833771
 [7] 0.003736942 0.003658015 0.003591711 0.003534745

[[4]]
 [1] 0.029348867 0.002399544 0.002151327 0.002031194 0.001951390 0.001892978
 [7] 0.001847542 0.001810690 0.001779888 0.001753562

[[5]]
 [1] 0.0203409831 0.0011978336 0.0010742352 0.0010145695 0.0009750065
 [6] 0.0009461006 0.0009236563 0.0009054852 0.0008903250 0.0008773910

It does not work at all -- in the end all esimates of `V` are very small and no signal is captured.

[A workaround implemented here](https://github.com/stephenslab/mmbr/blob/9e8701d4442c400909fd343fabb4d01f293f846d/R/bayesian_multivariate_regression.R#L101) avoids inverse on input prior when it is not invertable. The disadvantage here is that some quantities have to be recomputed which could otherwise be avoided using original direct inverse implementation.

### Verify the workaround agrees with direct method when prior is invertable

But before using this approach we need to establish that the result is the same as the original implementation when prior is invertable. To achieve this I use a diagonal prior matrix which is invertable. But to trigger the workaround code, I manually mess up [this line](https://github.com/stephenslab/mmbr/blob/9e8701d4442c400909fd343fabb4d01f293f846d/R/bayesian_multivariate_regression.R#L120) to take a non-existing function for inverse, so I can run the workaround even when prior is invertable.

In [12]:
V = diag(2)

In [ ]:
# This is the direct inverse version
devtools::load_all('~/GIT/software/mmbr')
fit1 = msusie(dat$X,dat$y,L=L,prior_variance=V, 
              compute_objective=T, estimate_residual_variance=F, 
              estimate_prior_variance=T, estimate_prior_method = 'EM',
              track_fit=TRUE)

In [15]:
fit1$V

[1] 0.3405679 0.0000000 0.0000000 0.0000000 0.0000000 0.0000000 0.0000000
 [8] 0.3452898 0.0000000 0.0000000

In [ ]:
# This is the indirect inverse version
# after manually messed up the version for direct inverse
devtools::load_all('~/GIT/software/mmbr')
fit2 = msusie(dat$X,dat$y,L=L,prior_variance=V, 
              compute_objective=T, estimate_residual_variance=F, 
              estimate_prior_variance=T, estimate_prior_method = 'EM',
              track_fit=TRUE)

In [18]:
fit2$V

[1] 0.3405679 0.0000000 0.0000000 0.0000000 0.0000000 0.0000000 0.0000000
 [8] 0.3452898 0.0000000 0.0000000

Let's try one more with the original simulated prior,

In [ ]:
# This is the direct inverse version
devtools::load_all('~/GIT/software/mmbr')
fit1 = msusie(dat$X,dat$y,L=L,prior_variance=dat$V, 
              compute_objective=T, estimate_residual_variance=F, 
              estimate_prior_variance=T, estimate_prior_method = 'EM',
              track_fit=TRUE)

In [20]:
fit1$V

[1] 0.7162723 0.0000000 0.0000000 0.0000000 0.0000000 0.0000000 0.0000000
 [8] 0.0000000 0.6946221 0.0000000

In [ ]:
# This is the indirect inverse version
# after manually messed up the version for direct inverse
devtools::load_all('~/GIT/software/mmbr')
fit2 = msusie(dat$X,dat$y,L=L,prior_variance=dat$V, 
              compute_objective=T, estimate_residual_variance=F, 
              estimate_prior_variance=T, estimate_prior_method = 'EM',
              track_fit=TRUE)

In [27]:
fit2$V

[1] 0.7162723 0.0000000 0.0000000 0.0000000 0.0000000 0.0000000 0.0000000
 [8] 0.0000000 0.6946221 0.0000000

It seems to work fine.

## Comparing the new EM implementation with `optim`

In [29]:
V = matrix(1,2,2)
devtools::load_all('~/GIT/software/mmbr')
fit3 = msusie(dat$X,dat$y,L=L,prior_variance=V, 
              compute_objective=T, estimate_residual_variance=F, 
              estimate_prior_variance=T, estimate_prior_method = 'EM',
              track_fit=TRUE)

Loading mmbr



In [31]:
fit3$elbo

[1] -198.1840 -194.8118 -192.2050 -190.0217 -188.3202 -187.1036 -186.3316
 [8] -185.8622 -185.5607 -185.3632 -185.0960 -184.7370 -184.7273 -184.7200
[15] -184.7141 -184.7095 -184.7058 -184.7029 -184.7005 -184.6986 -184.6970
[22] -184.6954 -184.6831 -184.6590 -184.6408 -184.6200 -184.5182 -184.4185
[29] -184.3525 -184.3170 -184.3001 -184.2925 -184.2892 -184.2877 -184.2871

In [32]:
fit3$V

[1] 0.000000 0.000000 0.000000 0.000000 0.000000 0.215164 0.000000 0.000000
 [9] 0.000000 0.000000

It seems to be working, and detects a signal with the 6th effect. 

To compare I also run `optim` directly,

In [34]:
fit4 = msusie(dat$X,dat$y,L=L,prior_variance=V,
              compute_objective=T, estimate_residual_variance=F, 
              estimate_prior_variance=T, estimate_prior_method = 'optim',
              track_fit=TRUE)

In [35]:
fit4$elbo

[1] -184.2867 -184.2867

In [36]:
fit4$V

[1] 0.2224369 0.0000000 0.0000000 0.0000000 0.0000000 0.0000000 0.0000000
 [8] 0.0000000 0.0000000 0.0000000

It takes less iterations (though not much less computational time, if not more) and the ELBO achieved is slightly higher.

An additional comparison between `EM` and `optim` [can be found here](em_optim_est_prior.html). `optim` seems better in that scenario.

## Verify that MASH prior is also correctly implemented

When `V` is not invertable,

In [ ]:
m_init = MashInitializer$new(list(V), 1)
fit5 = msusie(dat$X,dat$y,L=L,prior_variance=m_init, 
              compute_objective=T, estimate_residual_variance=F, precompute_covariances=T,
              estimate_prior_variance=T, estimate_prior_method = 'EM',
              track_fit=TRUE)

In [5]:
fit5$V

[1] 0.000000 0.000000 0.000000 0.000000 0.000000 0.215164 0.000000 0.000000
 [9] 0.000000 0.000000

When `V` is invertable,

In [27]:
m_init = MashInitializer$new(list(diag(2)), 1)
fit6 = msusie(dat$X,dat$y,L=L,prior_variance=m_init, 
              compute_objective=T, estimate_residual_variance=F, precompute_covariances=T,
              estimate_prior_variance=T, estimate_prior_method = 'EM',
              track_fit=TRUE)

In [8]:
fit6$V

[1] 0.3405679 0.0000000 0.0000000 0.0000000 0.0000000 0.0000000 0.0000000
 [8] 0.3452898 0.0000000 0.0000000

This result agrees with above from BMR. Now I comment out the line to precompute inverse so it will use the trick around inverse implemented for MASH,

In [28]:
devtools::load_all('~/GIT/software/mmbr')
m_init = MashInitializer$new(list(diag(2)), 1)
fit7 = msusie(dat$X,dat$y,L=L,prior_variance=m_init,
              compute_objective=T, estimate_residual_variance=F, precompute_covariances=T,
              estimate_prior_variance=T, estimate_prior_method = 'EM',
              track_fit=TRUE)

Loading mmbr



NULL


In [29]:
fit7$V

[1] 0.3405679 0.0000000 0.0000000 0.0000000 0.0000000 0.0000000 0.0000000
 [8] 0.3452898 0.0000000 0.0000000